In [ ]:
#Cell0: 特徴量設計
import os
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd


# =========================
# 重要パラメータブロック
# =========================

BASE_DIR = Path(r"C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ\研究\本実験結果")

SUBJECT_IDS = [
    "10061", "10063", "10064",
    "10071", "10072", "10073", "10074",
    "10081", "10082", "10083",
    "10091", "10092", "10093", "10094",
    "10101", "10102", "10103",
]

# 解析対象時間（秒）
T_START = 1770.0
T_END = 2400.0

# スライディング窓の設定（本研究）
WINDOW_SEC = 3           # 本研究: 特徴量窓幅 3秒
SLIDE_STEP_SEC = 1       # 本研究: 窓終端 t を 1秒刻みで計算
PC_LAG_SEC = 3           # 本研究: PCは「3秒前の窓」と比較（= 1つ前の3秒ブロック）

# 参考: 先行研究③（VRジェットコースター LSTM）
#   WINDOW_SEC   ≒ 3     # 同じく 3秒ローリング窓で rma/max/min/pc を計算
#   SLIDE_STEP_SEC ≒ 0.5 # 0.5秒刻みで特徴列 f(t) を作成（2 Hz の時系列）
#   シーケンス長  = 30   # 30ステップの入力系列 (= 過去 15秒分の f(t))
#   ラベル        = 1つ  # 各 15秒ブロックの終端時刻に対応する CS(0/1) を1つ付与

# pc の計算パラメータ
PC_DEFAULT_VALUE = 0.0   # 前平均が無い/極小のときに入れる値
PC_EPS = 1e-6            # 「ほぼゼロ判定」のしきい値

# 窓終端の t の範囲（整数秒）
T_MIN_OUT = int(T_START + WINDOW_SEC)   # 1770 + 3 = 1773
T_MAX_OUT = int(T_END)                  # 2400

# FMS_TEXT（与えられたものをそのまま使用）
FMS_TEXT: Dict[str, str] = {
    "10061": "0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 2 1",
    "10063": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0",
    "10064": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1",
    "10071": "0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 1 1 1 1 1",
    "10072": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0",
    "10073": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0",
    "10074": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0",
    "10081": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1",
    "10082": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1",
    "10083": "0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1",
    "10091": "0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1",
    "10092": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0",
    "10093": "0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 2 2 3 3 4 4",
    "10094": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2",
    "10101": "0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3 3",
    "10102": "0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 2 2 2 2 2",
    "10103": "0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 2 3 3 4",
}


# =========================
# ユーティリティ関数
# =========================

def minmax_scale(values: np.ndarray, channel_name: str, sid: str) -> np.ndarray:
    """[1770,2400]内の値をmin-maxスケーリング。min==maxまたは全部NaNなら全0."""
    arr = np.asarray(values, float)
    valid = np.isfinite(arr)
    if not valid.any():
        print(f"[WARN] {sid} {channel_name}: all NaN -> set all zeros")
        return np.zeros_like(arr, float)
    vmin = np.nanmin(arr[valid])
    vmax = np.nanmax(arr[valid])
    if vmax - vmin == 0:
        print(f"[WARN] {sid} {channel_name}: min==max ({vmin}) -> set all zeros")
        return np.zeros_like(arr, float)
    return (arr - vmin) / (vmax - vmin)


def compute_window_features_continuous(
    times: np.ndarray,
    values: np.ndarray,
    t_grid: np.ndarray,
    window_len: float,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    不均一サンプリング（1000Hz, 15Hzなど）について、
    各 t (窓終端) に対する [t-window_len, t] の平均/最大/最小を二重ポインタで計算。
    """
    times = np.asarray(times, float)
    values = np.asarray(values, float)
    n = len(times)
    means = np.full(t_grid.shape, np.nan, dtype=float)
    vmaxs = np.full(t_grid.shape, np.nan, dtype=float)
    vmins = np.full(t_grid.shape, np.nan, dtype=float)
    start = 0
    end = -1
    for i, t in enumerate(t_grid):
        w_start = t - window_len
        # 左端を前に進める
        while start < n and times[start] < w_start:
            start += 1
        # 右端を前に進める
        while end + 1 < n and times[end + 1] <= t:
            end += 1
        if end >= start and start < n:
            seg = values[start:end + 1]
            if seg.size > 0:
                valid = np.isfinite(seg)
                if valid.any():
                    segv = seg[valid]
                    means[i] = segv.mean()
                    vmaxs[i] = segv.max()
                    vmins[i] = segv.min()
    return means, vmaxs, vmins


def build_hr_1s_from_rr(
    r_times: np.ndarray,
    rr_intervals: np.ndarray,
    t_start: int,
    t_end: int,
    sid: str,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    RR時刻列から、t_start〜t_end の1秒グリッドHR_1s(t)を作る。
    ルール:
      区間 [R_k, R_{k+1}) にいる t のHRは HR_k = 60/RR_k

    ※ 有効RRがゼロ件なら即エラー（仕様4）
    """
    r_times = np.asarray(r_times, float)
    rr_intervals = np.asarray(rr_intervals, float)
    # 有効なRRだけ
    mask = np.isfinite(rr_intervals) & (rr_intervals > 0)
    if not mask.any():
        raise RuntimeError(f"[ERROR] {sid} RRtime: no valid RR_interval_sec (>0)")

    r_times = r_times[mask]
    rr_intervals = rr_intervals[mask]

    t_grid = np.arange(int(t_start), int(t_end) + 1)

    hr_k = 60.0 / rr_intervals
    # 時刻順
    order = np.argsort(r_times)
    r_times = r_times[order]
    hr_k = hr_k[order]

    hr_1s = np.full_like(t_grid, np.nan, dtype=float)
    k = 0
    n = len(r_times)
    for i, t in enumerate(t_grid):
        # 次のR波が t 以下なら進める
        while k + 1 < n and r_times[k + 1] <= t:
            k += 1
        if r_times[k] <= t:
            hr_1s[i] = hr_k[k]

    return t_grid, hr_1s


def compute_hr_features(
    hr_norm: np.ndarray,
    t_grid: np.ndarray,
    t_min: int,
    t_max: int,
    window_sec: int,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    HR_norm(1Hz) から、t=t_min..t_max について
    ・window_sec 秒分の rma
    ・window_sec 秒分の max/min
    ・PC_LAG_SEC 秒前の窓との pc
    を計算。
    """
    T0 = int(t_grid[0])
    mask = (t_grid >= t_min) & (t_grid <= t_max)
    t_out = t_grid[mask]
    n_out = len(t_out)

    rma = np.full(n_out, np.nan, float)
    vmax = np.full(n_out, np.nan, float)
    vmin = np.full(n_out, np.nan, float)
    pc = np.full(n_out, PC_DEFAULT_VALUE, float)

    # 1Hzなので「window_sec サンプル」を窓にする
    win_samples = int(window_sec)
    lag_sec = int(PC_LAG_SEC)

    # rma/max/min
    for i, t in enumerate(t_out):
        idx_end = int(t - T0)
        idx_start = idx_end - win_samples + 1
        if idx_start < 0:
            continue
        seg = hr_norm[idx_start: idx_end + 1]
        valid = np.isfinite(seg)
        if valid.any():
            segv = seg[valid]
            rma[i] = segv.mean()
            vmax[i] = segv.max()
            vmin[i] = segv.min()

    # pc（lag_sec 秒前の窓との変化率）
    for i, t in enumerate(t_out):
        if t - lag_sec < t_min:
            pc[i] = PC_DEFAULT_VALUE
            continue
        j = int((t - lag_sec) - t_min)  # t_out[j] = t - lag_sec
        prev = rma[j]
        cur = rma[i]
        if np.isfinite(prev) and abs(prev) > PC_EPS and np.isfinite(cur):
            pc[i] = (cur - prev) / prev
        else:
            pc[i] = PC_DEFAULT_VALUE

    return t_out, rma, vmax, vmin, pc


def forward_fill_to_1s(
    times: np.ndarray,
    values: np.ndarray,
    t_start: int,
    t_end: int,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    不規則（ここでは10秒刻み）な時系列を、t_start..t_end の1秒グリッドにforward fill。
    """
    times = np.asarray(times, float)
    values = np.asarray(values, float)
    order = np.argsort(times)
    times = times[order]
    values = values[order]

    t_grid = np.arange(int(t_start), int(t_end) + 1)
    # 各 t について、times <= t の最後のインデックスを searchsorted で取得
    idx = np.searchsorted(times, t_grid, side="right") - 1
    out = np.full_like(t_grid, np.nan, float)
    valid = idx >= 0
    out[valid] = values[idx[valid]]
    return t_grid, out


def compute_pc_from_mean(
    mean_arr: np.ndarray,
    t_grid: np.ndarray,
    t_min: int,
    t_max: int,
) -> np.ndarray:
    """3秒（PC_LAG_SEC秒）前の平均との変化率を、mean_arr(t)とt_gridから計算。"""
    mask = (t_grid >= t_min) & (t_grid <= t_max)
    t_out = t_grid[mask]
    pc = np.full(t_out.shape, PC_DEFAULT_VALUE, float)
    lag_sec = int(PC_LAG_SEC)

    for i, t in enumerate(t_out):
        if t - lag_sec < t_min:
            pc[i] = PC_DEFAULT_VALUE
            continue
        j = int((t - lag_sec) - t_min)
        prev = mean_arr[j]
        cur = mean_arr[i]
        if np.isfinite(prev) and abs(prev) > PC_EPS and np.isfinite(cur):
            pc[i] = (cur - prev) / prev
        else:
            pc[i] = PC_DEFAULT_VALUE
    return pc


def build_fms_series_for_t_grid(sid: str, t_grid: np.ndarray) -> np.ndarray:
    """FMS_TEXT から t_grid(整数秒) 用の FMS(t) を作る。"""
    text = FMS_TEXT[sid]
    fms_list = [int(x) for x in text.split()]
    if len(fms_list) != 21:
        raise ValueError(f"[FMS] {sid}: expected 21 values, got {len(fms_list)}")
    fms_arr = np.array(fms_list, int)
    out = np.zeros_like(t_grid, int)
    for i, t in enumerate(t_grid):
        idx = int((t - T_START) // 30)
        if idx < 0:
            idx = 0
        elif idx > 20:
            idx = 20
        out[i] = fms_arr[idx]
    return out


# =========================
# メイン処理
# =========================

def process_subject(sid: str) -> None:
    print(f"[INFO] Subject {sid} start")

    # ---- 入力パス ----
    offset_dir = BASE_DIR / sid / "OFFSET"
    feat_dir = BASE_DIR / sid / "FEATURE"
    out_dir = BASE_DIR / sid / "FEATURE2"
    out_dir.mkdir(parents=True, exist_ok=True)

    path_pulse = offset_dir / f"{sid}_Pulse.csv"
    path_sweat = offset_dir / f"{sid}_Sweat.csv"
    path_faceA = offset_dir / f"{sid}_FaceA.csv"
    path_faceB = offset_dir / f"{sid}_FaceB.csv"
    path_skinos = offset_dir / f"{sid}_Skinos.csv"
    path_rr = feat_dir / f"{sid}_RRtime.csv"

    # ---- 出力時刻グリッド（窓終端） ----
    t_out = np.arange(T_MIN_OUT, T_MAX_OUT + 1, int(SLIDE_STEP_SEC))  # 1773..2400

    # ---- FMS 列 ----
    fms_out = build_fms_series_for_t_grid(sid, t_out)

    # =====================
    # Pulse: 1000Hz  (仕様2: [1770,2400]にデータがなければ即エラー)
    # =====================
    df_pulse = pd.read_csv(path_pulse)
    df_pulse = df_pulse[(df_pulse["Time_sec"] >= T_START) & (df_pulse["Time_sec"] <= T_END)].copy()
    if df_pulse.empty:
        raise RuntimeError(f"[ERROR] {sid} Pulse: no data in [{T_START}, {T_END}]")
    df_pulse = df_pulse.sort_values("Time_sec")
    pulse_norm = minmax_scale(df_pulse["Pulse"].to_numpy(), "Pulse", sid)
    times_pulse = df_pulse["Time_sec"].to_numpy()
    pulse_mean3, pulse_max3, pulse_min3 = compute_window_features_continuous(
        times_pulse, pulse_norm, t_out, window_len=WINDOW_SEC
    )
    pulse_pc3 = compute_pc_from_mean(pulse_mean3, t_out, T_MIN_OUT, T_MAX_OUT)

    # =====================
    # Sweat (GSR): 1000Hz  (仕様2)
    # =====================
    df_sweat = pd.read_csv(path_sweat)
    df_sweat = df_sweat[(df_sweat["Time_sec"] >= T_START) & (df_sweat["Time_sec"] <= T_END)].copy()
    if df_sweat.empty:
        raise RuntimeError(f"[ERROR] {sid} Sweat: no data in [{T_START}, {T_END}]")
    df_sweat = df_sweat.sort_values("Time_sec")
    gsr_norm = minmax_scale(df_sweat["Sweat"].to_numpy(), "Sweat", sid)
    times_gsr = df_sweat["Time_sec"].to_numpy()
    gsr_mean3, gsr_max3, gsr_min3 = compute_window_features_continuous(
        times_gsr, gsr_norm, t_out, window_len=WINDOW_SEC
    )
    gsr_pc3 = compute_pc_from_mean(gsr_mean3, t_out, T_MIN_OUT, T_MAX_OUT)

    # =====================
    # FaceA/B: 15Hz  (仕様2 & 仕様3)
    # =====================
    df_faceA = pd.read_csv(path_faceA)
    df_faceB = pd.read_csv(path_faceB)
    df_faceA = df_faceA[(df_faceA["Time_sec"] >= T_START) & (df_faceA["Time_sec"] <= T_END)].copy()
    df_faceB = df_faceB[(df_faceB["Time_sec"] >= T_START) & (df_faceB["Time_sec"] <= T_END)].copy()

    if df_faceA.empty:
        raise RuntimeError(f"[ERROR] {sid} FaceA: no data in [{T_START}, {T_END}]")
    if df_faceB.empty:
        raise RuntimeError(f"[ERROR] {sid} FaceB: no data in [{T_START}, {T_END}]")

    df_faceA = df_faceA.sort_values("Time_sec").reset_index(drop=True)
    df_faceB = df_faceB.sort_values("Time_sec").reset_index(drop=True)

    # ---- 仕様3: FaceA/B の長さ＆Time_sec一致チェック ----
    if len(df_faceA) != len(df_faceB):
        raise RuntimeError(f"[ERROR] {sid} FaceA/FaceB length mismatch: "
                           f"{len(df_faceA)} vs {len(df_faceB)}")
    if not np.allclose(df_faceA["Time_sec"].to_numpy(),
                       df_faceB["Time_sec"].to_numpy()):
        raise RuntimeError(f"[ERROR] {sid} FaceA/FaceB Time_sec mismatch")

    # 完全一致前提で結合
    df_face = pd.DataFrame({
        "Time_sec": df_faceA["Time_sec"].to_numpy(),
        "FaceA_BoxAve": df_faceA["FaceA_BoxAve"].to_numpy(),
        "FaceB_BoxAve": df_faceB["FaceB_BoxAve"].to_numpy(),
    })

    # 簡易1秒ローパス（移動平均：15サンプル窓, center=True）
    sA = df_face["FaceA_BoxAve"].astype(float)
    sB = df_face["FaceB_BoxAve"].astype(float)
    faceA_lp = sA.rolling(window=15, center=True, min_periods=1).mean().to_numpy()
    faceB_lp = sB.rolling(window=15, center=True, min_periods=1).mean().to_numpy()

    # IQRベース外れ値除去＋線形補間
    def clean_face_channel(arr: np.ndarray) -> np.ndarray:
        x = arr.astype(float).copy()
        med = np.nanmedian(x)
        q1 = np.nanpercentile(x, 25)
        q3 = np.nanpercentile(x, 75)
        iqr = q3 - q1
        if iqr <= 0:
            s = pd.Series(x)
            return s.interpolate(method="linear", limit_direction="both").to_numpy()
        lower = med - 3 * iqr
        upper = med + 3 * iqr
        mask_out = (x < lower) | (x > upper)
        x[mask_out] = np.nan
        s = pd.Series(x)
        x_filled = s.interpolate(method="linear", limit_direction="both").to_numpy()
        return x_filled

    faceA_clean = clean_face_channel(faceA_lp)
    faceB_clean = clean_face_channel(faceB_lp)

    # min-max正規化
    faceA_norm = minmax_scale(faceA_clean, "FaceA_BoxAve", sid)
    faceB_norm = minmax_scale(faceB_clean, "FaceB_BoxAve", sid)

    # 和＆差
    face_sum = faceA_norm + faceB_norm
    face_diff = faceB_norm - faceA_norm
    times_face = df_face["Time_sec"].to_numpy()

    # 3秒窓特徴（平均のみ使用）
    face_sum_mean3, _, _ = compute_window_features_continuous(
        times_face, face_sum, t_out, window_len=WINDOW_SEC
    )
    face_diff_mean3, _, _ = compute_window_features_continuous(
        times_face, face_diff, t_out, window_len=WINDOW_SEC
    )
    face_sum_pc3 = compute_pc_from_mean(face_sum_mean3, t_out, T_MIN_OUT, T_MAX_OUT)
    face_diff_pc3 = compute_pc_from_mean(face_diff_mean3, t_out, T_MIN_OUT, T_MAX_OUT)

    # =====================
    # RRtime -> HR_1s -> HR 3秒窓特徴  (仕様2 & 仕様4)
    # =====================
    df_rr = pd.read_csv(path_rr)
    if df_rr.empty:
        raise RuntimeError(f"[ERROR] {sid} RRtime: file is empty")

    # 仕様2: [1770,2400] に R波が1つもない場合はエラー
    df_rr_win = df_rr[(df_rr["Time_sec"] >= T_START) & (df_rr["Time_sec"] <= T_END)]
    if df_rr_win.empty:
        raise RuntimeError(f"[ERROR] {sid} RRtime: no R waves in [{T_START}, {T_END}]")

    r_times = df_rr["Time_sec"].to_numpy()
    rr_int = df_rr["RR_interval_sec"].to_numpy()
    t_grid_hr, hr_1s = build_hr_1s_from_rr(r_times, rr_int, int(T_START), int(T_END), sid)

    # [1770,2400]だけ抜き出してmin-max
    hr_norm = minmax_scale(hr_1s, "HR_1s", sid)
    # HR 3秒窓特徴（1Hz）
    t_hr_out, hr_rma3, hr_max3, hr_min3, hr_pc3 = compute_hr_features(
        hr_norm, t_grid_hr, T_MIN_OUT, T_MAX_OUT, window_sec=WINDOW_SEC
    )
    # t_hr_out と t_out は同じはず
    assert np.array_equal(t_hr_out, t_out), "[BUG] HR t_grid mismatch"

    # =====================
    # Skinos: 10秒刻み -> 1秒刻み  (仕様2)
    # =====================
    df_skin = pd.read_csv(path_skinos)
    df_skin = df_skin[(df_skin["Time_sec"] >= T_START) & (df_skin["Time_sec"] <= T_END)].copy()
    if df_skin.empty:
        raise RuntimeError(f"[ERROR] {sid} Skinos: no data in [{T_START}, {T_END}]")
    df_skin = df_skin.sort_values("Time_sec")

    skin_sweat_norm = minmax_scale(df_skin["Sweat_Rate"].to_numpy(), "Skinos_Sweat_Rate", sid)
    skin_hr_norm = minmax_scale(df_skin["Heart_Rate"].to_numpy(), "Skinos_Heart_Rate", sid)
    skin_temp_norm = minmax_scale(df_skin["Skin_Temp"].to_numpy(), "Skinos_Skin_Temp", sid)

    times_skin = df_skin["Time_sec"].to_numpy()
    t_grid_skin, skin_sweat_1s = forward_fill_to_1s(times_skin, skin_sweat_norm, int(T_START), int(T_END))
    _, skin_hr_1s = forward_fill_to_1s(times_skin, skin_hr_norm, int(T_START), int(T_END))
    _, skin_temp_1s = forward_fill_to_1s(times_skin, skin_temp_norm, int(T_START), int(T_END))

    # t_grid_skin は1770..2400なので、t_out(1773..2400)に合わせて切り出し
    mask_skin = (t_grid_skin >= T_MIN_OUT) & (t_grid_skin <= T_MAX_OUT)
    assert np.array_equal(t_grid_skin[mask_skin], t_out), "[BUG] Skinos t_grid mismatch"
    skin_sweat_out = skin_sweat_1s[mask_skin]
    skin_hr_out = skin_hr_1s[mask_skin]
    skin_temp_out = skin_temp_1s[mask_skin]

    # =====================
    # DataFrame にまとめて保存
    # =====================
    df_out = pd.DataFrame({
        "Time_sec": t_out.astype(int),
        "FMS": fms_out.astype(int),
        "Pulse_rma3": pulse_mean3,
        "Pulse_max3": pulse_max3,
        "Pulse_min3": pulse_min3,
        "Pulse_pc3": pulse_pc3,
        "HR_rma3": hr_rma3,
        "HR_max3": hr_max3,
        "HR_min3": hr_min3,
        "HR_pc3": hr_pc3,
        "GSR_rma3": gsr_mean3,
        "GSR_max3": gsr_max3,
        "GSR_min3": gsr_min3,
        "GSR_pc3": gsr_pc3,
        "FaceSum_mean3": face_sum_mean3,
        "FaceDiff_mean3": face_diff_mean3,
        "FaceSum_pc3": face_sum_pc3,
        "FaceDiff_pc3": face_diff_pc3,
        "Skinos_SweatRate": skin_sweat_out,
        "Skinos_HeartRate": skin_hr_out,
        "Skinos_SkinTemp": skin_temp_out,
    })

    out_path = out_dir / f"{sid}_3sFeat_1sSlide.csv"
    df_out.to_csv(out_path, index=False)
    print(f"[INFO] Subject {sid} done -> {out_path}")


if __name__ == "__main__":
    for sid in SUBJECT_IDS:
        process_subject(sid)


In [ ]:
#Cell1-LSTM: 3秒窓特徴量LSTM（LOSO＋ROC-AUC）

import os
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score


# -----------------------------
# パス・基本設定
# -----------------------------
BASE_DIR = Path(r"C:\Users\taiki\OneDrive - Science Tokyo\デスクトップ\研究\本実験結果")

SUBJECT_IDS = [
    "10061", "10063", "10064",
    "10071", "10072", "10073", "10074",
    "10081", "10082", "10083",
    "10091", "10092", "10093", "10094",
    "10101", "10102", "10103",
]

# このCell用の出力ディレクトリ
CELL_NAME = "Cell1-LSTM"
OUT_DIR = BASE_DIR / "解析" / "Cell1"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"[INFO] Cell: {CELL_NAME}, OUT_DIR = {OUT_DIR}")

# -----------------------------
# 時間・シーケンス仕様
# -----------------------------
WINDOW_SEC = 3          # 3秒窓（既にFEATURE2で反映済み）
SLIDE_STEP_SEC = 1      # 1秒刻み（既にFEATURE2で反映済み）

# LSTM に入れる過去ステップ数（= 過去 SEQ_LEN 秒分）
SEQ_LEN = 30

# FEATURE2 での最初の出力時刻（T_START+WINDOW_SEC = 1770+3）
BASE_T_MIN = 1773

# ターゲットの最小時刻：最初の出力時刻＋(SEQ_LEN-1)
# 例：BASE_T_MIN=1773, SEQ_LEN=30 → 1773+29 = 1802
TARGET_T_MIN = BASE_T_MIN + (SEQ_LEN - 1)
TARGET_T_MAX = 2400     # 上限はこれまで通り 2400 秒

# ラベル閾値：FMS >= 1 を陽性とする
FMS_POS_THRESHOLD = 1

# -----------------------------
# LSTMハイパラ（変更候補は CSV に出力）
# -----------------------------
HIDDEN_SIZE = 32
FC_HIDDEN_SIZE = 8
DROPOUT_LSTM = 0.0
DROPOUT_FC = 0.5
LEARNING_RATE = 0.005
BATCH_SIZE = 256
N_EPOCHS = 10
WEIGHT_DECAY = 1e-4  # L2正則化（Adam の weight_decay）

# -----------------------------
# 特徴量ON/OFF設定
# -----------------------------
FEATURE_SWITCHES: List[Tuple[str, bool]] = [
    ("Pulse_rma3",       True),
    ("Pulse_max3",       True),
    ("Pulse_min3",       True),
    ("Pulse_pc3",        True),
    ("HR_rma3",          True),
    ("HR_max3",          True),
    ("HR_min3",          True),
    ("HR_pc3",           True),
    ("GSR_rma3",         True),
    ("GSR_max3",         True),
    ("GSR_min3",         True),
    ("GSR_pc3",          True),
    ("FaceSum_mean3",    True),
    ("FaceDiff_mean3",   True),
    ("FaceSum_pc3",      True),
    ("FaceDiff_pc3",     True),
    ("Skinos_SweatRate", True),
    ("Skinos_HeartRate", True),
    ("Skinos_SkinTemp",  True),
]

FEATURE_COLS: List[str] = [name for name, use in FEATURE_SWITCHES if use]
if len(FEATURE_COLS) == 0:
    raise RuntimeError("[ERROR] FEATURE_SWITCHES: 有効な特徴量が0個です（すべてFalse）。")

N_FEATURES = len(FEATURE_COLS)
print(f"[INFO] Using {N_FEATURES} features:", ", ".join(FEATURE_COLS))


# -----------------------------
# LSTM モデル定義
# -----------------------------
class LSTMMotionSickness(nn.Module):
    """
    単方向1層LSTM → Dropout → FC(HIDDEN_SIZE→FC_HIDDEN_SIZE) → ReLU → FC → ロジット
    出力はロジット（Sigmoidはloss/評価側で適用）
    """
    def __init__(
        self,
        input_size: int,
        hidden_size: int = HIDDEN_SIZE,
        fc_hidden_size: int = FC_HIDDEN_SIZE,
        dropout_lstm: float = DROPOUT_LSTM,
        dropout_fc: float = DROPOUT_FC,
    ):
        super().__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=False,
            dropout=dropout_lstm,
        )
        self.dropout = nn.Dropout(dropout_fc)
        self.fc1 = nn.Linear(hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc_out = nn.Linear(fc_hidden_size, 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch, seq_len, input_size)
        return: ロジット (batch,)
        """
        lstm_out, (hn, cn) = self.lstm(x)
        h_last = hn[-1]              # (batch, hidden_size)
        z = self.dropout(h_last)
        z = self.relu(self.fc1(z))
        z = self.fc_out(z)           # (batch, 1)
        return z.squeeze(-1)         # (batch,)


# -----------------------------
# データ読み込み & シーケンス生成
# -----------------------------
def load_subject_df(sid: str) -> pd.DataFrame:
    """FEATURE2/{sid}_3sFeat_1sSlide.csv を読み込む."""
    path = BASE_DIR / sid / "FEATURE2" / f"{sid}_3sFeat_1sSlide.csv"
    if not path.exists():
        raise FileNotFoundError(f"[ERROR] Subject {sid}: file not found: {path}")
    df = pd.read_csv(path)
    df = df.sort_values("Time_sec").reset_index(drop=True)
    return df


def build_sequences_for_subject(
    sid: str,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    1被験者について:
      - FEATURE2 CSVを読み込み
      - FMS>=1 を陽性にした y(t) を作成
      - t=TARGET_T_MIN〜TARGET_T_MAX の各時刻 t に対し，
          X_seq(t) = [t-SEQ_LEN+1 .. t] のシーケンスを生成
      - その際，特徴量内にNaNがあれば即エラー
    """
    df = load_subject_df(sid)

    # 必要列が揃っているかチェック
    required_cols = ["Time_sec", "FMS"] + FEATURE_COLS
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise RuntimeError(f"[ERROR] Subject {sid}: missing columns in FEATURE2 csv: {missing}")

    # NaNチェック（仕様：NaNがあれば即エラー）
    if df[FEATURE_COLS].isna().values.any():
        nan_mask = df[FEATURE_COLS].isna()
        bad_idx = np.where(nan_mask.values)[0][0]
        bad_time = df.loc[bad_idx, "Time_sec"]
        bad_cols = list(nan_mask.columns[nan_mask.iloc[bad_idx]])
        raise RuntimeError(
            f"[ERROR] Subject {sid}: NaN detected at Time_sec={bad_time}, cols={bad_cols}"
        )

    times = df["Time_sec"].to_numpy().astype(int)
    fms = df["FMS"].to_numpy().astype(int)
    features = df[FEATURE_COLS].to_numpy().astype(np.float32)

    # TARGET_T_MIN〜TARGET_T_MAX の範囲があるか
    target_mask = (times >= TARGET_T_MIN) & (times <= TARGET_T_MAX)
    if not target_mask.any():
        raise RuntimeError(f"[ERROR] Subject {sid}: no Time_sec in [{TARGET_T_MIN}, {TARGET_T_MAX}]")

    X_list: List[np.ndarray] = []
    y_list: List[int] = []
    t_list: List[int] = []
    fms_list: List[int] = []

    for idx in range(len(times)):
        t = times[idx]
        if t < TARGET_T_MIN or t > TARGET_T_MAX:
            continue

        if idx < SEQ_LEN - 1:
            raise RuntimeError(
                f"[ERROR] Subject {sid}: idx={idx}, Time_sec={t} has no enough history (need {SEQ_LEN})."
            )

        window_feat = features[idx - SEQ_LEN + 1: idx + 1, :]  # (SEQ_LEN, N_FEATURES)
        if not np.isfinite(window_feat).all():
            raise RuntimeError(
                f"[ERROR] Subject {sid}: non-finite value in sequence ending at Time_sec={t}"
            )

        # ラベル：FMS>=1
        y = 1 if fms[idx] >= FMS_POS_THRESHOLD else 0

        X_list.append(window_feat)
        y_list.append(y)
        t_list.append(t)
        fms_list.append(int(fms[idx]))

    X_seq = np.stack(X_list).astype(np.float32)   # (N_seq, SEQ_LEN, N_FEATURES)
    y_seq = np.array(y_list, dtype=np.int64)
    t_seq = np.array(t_list, dtype=np.int64)
    fms_seq = np.array(fms_list, dtype=np.int64)

    print(
        f"[INFO] Subject {sid}: target Time_sec range = {t_seq[0]}–{t_seq[-1]}, "
        f"N_seq = {len(t_seq)}, N_pos = {y_seq.sum()}, N_neg = {len(y_seq) - y_seq.sum()}"
    )

    return X_seq, y_seq, t_seq, fms_seq


# -----------------------------
# LOSO 学習・評価ループ
# -----------------------------
def train_one_fold(
    train_X: np.ndarray,
    train_y: np.ndarray,
    device: torch.device,
) -> Tuple[LSTMMotionSickness, List[float]]:
    """
    1つのLOSO foldについて，訓練データのみを使ってLSTMを学習する。
    戻り値: (学習済みモデル, 各epochの平均train lossリスト)
    """
    model = LSTMMotionSickness(input_size=N_FEATURES).to(device)
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
    )
    criterion = nn.BCEWithLogitsLoss()

    # 陽性割合をプリント
    n_train = len(train_y)
    n_pos = int(train_y.sum())
    n_neg = n_train - n_pos
    pos_ratio = n_pos / n_train if n_train > 0 else 0.0
    print(
        f"[INFO] Train stats: N={n_train}, N_pos={n_pos}, N_neg={n_neg}, "
        f"pos_ratio={pos_ratio:.3f}"
    )

    # DataLoader 構築
    X_tensor = torch.from_numpy(train_X).float()
    y_tensor = torch.from_numpy(train_y).float()
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    epoch_loss_list: List[float] = []

    for epoch in range(1, N_EPOCHS + 1):
        model.train()
        running_loss = 0.0
        n_batches = 0

        for batch_X, batch_y in loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            logits = model(batch_X)           # (batch,)
            loss = criterion(logits, batch_y) # BCEWithLogitsLoss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            n_batches += 1

        avg_loss = running_loss / max(n_batches, 1)
        epoch_loss_list.append(avg_loss)

        if epoch % 5 == 0 or epoch == 1 or epoch == N_EPOCHS:
            print(f"[INFO] Epoch {epoch:02d}/{N_EPOCHS} - train_loss={avg_loss:.4f}")

    return model, epoch_loss_list


def main():
    # デバイス選択
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[INFO] Using device: {device}")

    # 再現性のためにseed固定
    torch.manual_seed(20251206)
    np.random.seed(20251206)

    # ---- 全被験者のシーケンスを構築 ----
    X_by_sid: Dict[str, np.ndarray] = {}
    y_by_sid: Dict[str, np.ndarray] = {}
    t_by_sid: Dict[str, np.ndarray] = {}
    fms_by_sid: Dict[str, np.ndarray] = {}

    for sid in SUBJECT_IDS:
        print(f"[INFO] ==== Build sequences: Subject {sid} ====")
        X_seq, y_seq, t_seq, fms_seq = build_sequences_for_subject(sid)
        X_by_sid[sid] = X_seq
        y_by_sid[sid] = y_seq
        t_by_sid[sid] = t_seq
        fms_by_sid[sid] = fms_seq

    all_y_tmp = np.concatenate([y_by_sid[sid] for sid in SUBJECT_IDS])
    print(
        f"[INFO] Overall (all subjects) target stats: "
        f"N={len(all_y_tmp)}, N_pos={all_y_tmp.sum()}, "
        f"pos_ratio={all_y_tmp.mean():.3f}"
    )

    # ---- LOSO 学習・評価 ----
    all_probs: List[np.ndarray] = []
    all_true: List[np.ndarray] = []
    pred_rows: List[pd.DataFrame] = []
    fold_summary_rows: List[Dict] = []
    epoch_loss_records: List[Dict] = []

    for test_sid in SUBJECT_IDS:
        print(f"\n[INFO] ===== LOSO fold: Test Subject {test_sid} =====")

        # 学習・テスト分割
        train_X_list = []
        train_y_list = []
        for sid in SUBJECT_IDS:
            if sid == test_sid:
                continue
            train_X_list.append(X_by_sid[sid])
            train_y_list.append(y_by_sid[sid])

        train_X = np.concatenate(train_X_list, axis=0)
        train_y = np.concatenate(train_y_list, axis=0)
        test_X = X_by_sid[test_sid]
        test_y = y_by_sid[test_sid]
        test_t = t_by_sid[test_sid]
        test_fms = fms_by_sid[test_sid]

        print(
            f"[INFO] Fold data sizes: "
            f"Train N_seq={len(train_y)}, Test N_seq={len(test_y)}"
        )

        # 1 fold 学習
        model, epoch_loss_list = train_one_fold(train_X, train_y, device=device)

        # epochごとの loss をログ用に保存
        for ep_idx, loss_val in enumerate(epoch_loss_list, start=1):
            epoch_loss_records.append(
                {
                    "SubjectID": test_sid,
                    "Epoch": ep_idx,
                    "TrainLoss": loss_val,
                }
            )

        # テスト被験者の予測確率
        model.eval()
        with torch.no_grad():
            X_test_tensor = torch.from_numpy(test_X).float().to(device)
            logits = model(X_test_tensor)
            probs = torch.sigmoid(logits).cpu().numpy()  # (N_test,)

        all_probs.append(probs)
        all_true.append(test_y.astype(int))

        # foldごとのROC-AUC
        n_pos_test = int(test_y.sum())
        n_neg_test = int(len(test_y) - n_pos_test)
        if n_pos_test == 0 or n_neg_test == 0:
            rocauc_fold = float("nan")
            print(
                f"[INFO] Subject {test_sid}: ROC-AUC undefined (N_pos={n_pos_test}, N_neg={n_neg_test})"
            )
        else:
            rocauc_fold = roc_auc_score(test_y, probs)
            print(
                f"[INFO] Subject {test_sid}: ROC-AUC(test fold) = {rocauc_fold:.4f} "
                f"(N_test={len(test_y)}, N_pos={n_pos_test}, N_neg={n_neg_test})"
            )

        fold_summary_rows.append(
            {
                "SubjectID": test_sid,
                "N_test": int(len(test_y)),
                "N_pos_test": n_pos_test,
                "N_neg_test": n_neg_test,
                "pos_ratio_test": float(test_y.mean()),
                "ROC_AUC_test": rocauc_fold,
            }
        )

        # このfoldの予測詳細
        df_fold = pd.DataFrame(
            {
                "SubjectID": test_sid,
                "Time_sec": test_t,
                "FMS": test_fms,
                "Label_bin": test_y.astype(int),
                "Prob_FMS_ge1": probs,
            }
        )
        pred_rows.append(df_fold)

    # ---- 全foldをまとめた ROC-AUC ----
    y_all = np.concatenate(all_true)
    p_all = np.concatenate(all_probs)

    n_total = len(y_all)
    n_pos = int(y_all.sum())
    n_neg = n_total - n_pos
    pos_ratio = n_pos / n_total if n_total > 0 else 0.0

    print("\n[INFO] ===== Overall LOSO result =====")
    print(
        f"[INFO] All folds combined: N={n_total}, N_pos={n_pos}, "
        f"N_neg={n_neg}, pos_ratio={pos_ratio:.3f}"
    )

    if n_pos == 0 or n_pos == n_total:
        raise RuntimeError(
            f"[ERROR] ROC-AUC undefined: labels are all the same "
            f"(N={n_total}, N_pos={n_pos})."
        )

    rocauc = roc_auc_score(y_all, p_all)
    print(f"[RESULT] Global ROC-AUC (LOSO, LSTM, FMS>=1) = {rocauc:.4f}")

    # ---- 結果保存 ----
    # 1) ROC-AUC のサマリ（ハイパラ込み）
    result_path = OUT_DIR / "Cell1_LSTM_LOSO_ROCAUC.csv"
    df_result = pd.DataFrame(
        {
            "ROC_AUC_global": [rocauc],
            "N_total": [n_total],
            "N_pos": [n_pos],
            "N_neg": [n_neg],
            "pos_ratio": [pos_ratio],
            "N_features": [N_FEATURES],
            "feature_list": [",".join(FEATURE_COLS)],
            # 変更候補ハイパラを全部記録
            "WINDOW_SEC": [WINDOW_SEC],
            "SLIDE_STEP_SEC": [SLIDE_STEP_SEC],
            "SEQ_LEN": [SEQ_LEN],
            "HIDDEN_SIZE": [HIDDEN_SIZE],
            "FC_HIDDEN_SIZE": [FC_HIDDEN_SIZE],
            "DROPOUT_LSTM": [DROPOUT_LSTM],
            "DROPOUT_FC": [DROPOUT_FC],
            "LEARNING_RATE": [LEARNING_RATE],
            "BATCH_SIZE": [BATCH_SIZE],
            "N_EPOCHS": [N_EPOCHS],
            "WEIGHT_DECAY": [WEIGHT_DECAY],
        }
    )
    df_result.to_csv(result_path, index=False)
    print(f"[INFO] Saved ROC-AUC result to: {result_path}")

    # 2) シーケンスごとの詳細予測
    df_pred = pd.concat(pred_rows, ignore_index=True)
    pred_path = OUT_DIR / "Cell1_LSTM_LOSO_pred_detail.csv"
    df_pred.to_csv(pred_path, index=False)
    print(f"[INFO] Saved per-sequence predictions to: {pred_path}")

    # 3) foldごとの summary（被験者別 ROC-AUC）
    df_fold_summary = pd.DataFrame(fold_summary_rows)
    fold_summary_path = OUT_DIR / "Cell1_LSTM_LOSO_fold_summary.csv"
    df_fold_summary.to_csv(fold_summary_path, index=False)
    print(f"[INFO] Saved per-fold summary to: {fold_summary_path}")

    # 4) epochごとの train loss
    df_loss = pd.DataFrame(epoch_loss_records)
    loss_path = OUT_DIR / "Cell1_LSTM_LOSO_train_loss_by_epoch.csv"
    df_loss.to_csv(loss_path, index=False)
    print(f"[INFO] Saved train loss by epoch to: {loss_path}")

    # ---- 最後に、被験者ごとのROC-AUCを()付きでプリント ----
    print("\n[SUMMARY] ===== Per-subject ROC-AUC (LOSO) =====")
    print(f"[SUMMARY] Global ROC-AUC (all folds combined) = {rocauc:.4f}")

    good_mask = df_fold_summary["ROC_AUC_test"].notna() & (df_fold_summary["ROC_AUC_test"] > 0.5)
    bad_mask = df_fold_summary["ROC_AUC_test"].notna() & (df_fold_summary["ROC_AUC_test"] <= 0.5)
    nan_mask = df_fold_summary["ROC_AUC_test"].isna()

    def format_sid_list(mask) -> str:
        rows = df_fold_summary.loc[mask, ["SubjectID", "ROC_AUC_test"]]
        if rows.empty:
            return "なし"
        return ", ".join(f"{row.SubjectID}({row.ROC_AUC_test:.3f})" for _, row in rows.iterrows())

    good_str = format_sid_list(good_mask)
    bad_str = format_sid_list(bad_mask)
    nan_sids = df_fold_summary.loc[nan_mask, "SubjectID"].tolist()
    nan_str = ", ".join(nan_sids) if len(nan_sids) > 0 else "なし"

    print(f"[SUMMARY] よく当たっている被験者(>0.5): {good_str}")
    print(f"[SUMMARY] あまり当たっていない被験者(<=0.5): {bad_str}")
    print(f"[SUMMARY] 評価不能(ROC-AUC算出不可): {nan_str}")


if __name__ == "__main__":
    main()


In [ ]:
#Cell2-LSTM-SHAP: LSTM（30秒履歴, 19特徴量）のSHAP重要度可視化#

import os
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import shap
import matplotlib.pyplot as plt


# --------------------------------
# 前提チェック（Cell1-LSTM 実行済み想定）
# --------------------------------
required_globals = [
    "BASE_DIR",
    "SUBJECT_IDS",
    "FEATURE_COLS",
    "SEQ_LEN",
    "TARGET_T_MIN",
    "TARGET_T_MAX",
    "FMS_POS_THRESHOLD",
    "LSTMMotionSickness",
    "build_sequences_for_subject",
    "HIDDEN_SIZE",
    "FC_HIDDEN_SIZE",
    "DROPOUT_LSTM",
    "DROPOUT_FC",
    "LEARNING_RATE",
    "BATCH_SIZE",
    "N_EPOCHS",
]
missing = [name for name in required_globals if name not in globals()]
if missing:
    raise RuntimeError(
        "[Cell2-LSTM-SHAP] 必要な定義が見つかりません。"
        "先に Cell1-LSTM を同じノートブック上で実行してください。\n"
        f"不足: {missing}"
    )

N_FEATURES = len(FEATURE_COLS)

# このCell用の出力ディレクトリ
CELL_NAME = "Cell2-LSTM-SHAP"
OUT_DIR = BASE_DIR / "解析" / "Cell2"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"[INFO][SHAP] Cell: {CELL_NAME}, OUT_DIR = {OUT_DIR}")

# SHAP用のサンプル数設定
N_BACKGROUND_MAX = 200   # 各foldで DeepExplainer の背景に使う最大サンプル数
N_SHAP_EVAL_MAX = 2000   # 各foldで SHAP を計算する最大サンプル数（訓練シーケンス）


# --------------------------------
# SHAP用ラッパーモデル
#   base_model: (batch, seq_len, feat) -> (batch,)
#   → DeepExplainer 用に (batch, 1) に変形
# --------------------------------
class WrappedLSTMForSHAP(nn.Module):
    """
    SHAP用ラッパー:
    - base_model の出力 (batch,) を (batch, 1) に変形して返す
    """
    def __init__(self, base_model: nn.Module):
        super().__init__()
        self.base_model = base_model

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logits = self.base_model(x)      # (batch,)
        return logits.unsqueeze(1)       # (batch, 1)


# -----------------------------
# 1 fold 学習（Cell1と同仕様）
# -----------------------------
def train_one_fold_for_shap(
    train_X: np.ndarray,
    train_y: np.ndarray,
    device: torch.device,
) -> "LSTMMotionSickness":
    """
    SHAP用：Cell1-LSTM と同じ設定で1fold分のLSTMを学習する。
    """
    model = LSTMMotionSickness(input_size=N_FEATURES).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.BCEWithLogitsLoss()

    n_train = len(train_y)
    n_pos = int(train_y.sum())
    n_neg = n_train - n_pos
    pos_ratio = n_pos / n_train if n_train > 0 else 0.0
    print(
        f"[INFO][SHAP] Train stats: N={n_train}, N_pos={n_pos}, N_neg={n_neg}, "
        f"pos_ratio={pos_ratio:.3f}"
    )

    X_tensor = torch.from_numpy(train_X).float()
    y_tensor = torch.from_numpy(train_y).float()
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    for epoch in range(1, N_EPOCHS + 1):
        model.train()
        running_loss = 0.0
        n_batches = 0

        for batch_X, batch_y in loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()
            logits = model(batch_X)
            loss = criterion(logits, batch_y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            n_batches += 1

        avg_loss = running_loss / max(n_batches, 1)
        if epoch % 5 == 0 or epoch == 1 or epoch == N_EPOCHS:
            print(f"[INFO][SHAP] Epoch {epoch:02d}/{N_EPOCHS} - train_loss={avg_loss:.4f}")

    return model


# -----------------------------
# メイン処理（LOSO＋SHAP）
# -----------------------------
def main():
    # デバイス選択
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[INFO][SHAP] Using device: {device}")

    torch.manual_seed(20251206)
    np.random.seed(20251206)

    # --- 全被験者のシーケンス構築（Cell1と同じ関数を再利用） ---
    X_by_sid: Dict[str, np.ndarray] = {}
    y_by_sid: Dict[str, np.ndarray] = {}

    for sid in SUBJECT_IDS:
        print(f"[INFO][SHAP] ==== Build sequences: Subject {sid} ====")
        X_seq, y_seq, t_seq, fms_seq = build_sequences_for_subject(sid)
        X_by_sid[sid] = X_seq
        y_by_sid[sid] = y_seq

    # 全体の陽性割合（参考）
    all_y_tmp = np.concatenate([y_by_sid[sid] for sid in SUBJECT_IDS])
    print(
        f"[INFO][SHAP] Overall (all subjects) target stats: "
        f"N={len(all_y_tmp)}, N_pos={all_y_tmp.sum()}, "
        f"pos_ratio={all_y_tmp.mean():.3f}"
    )

    # --- LOSO 各foldで SHAP を計算 ---
    fold_importances: List[np.ndarray] = []
    fold_meta: List[Dict[str, str]] = []

    # beeswarm 用：各foldの shap と特徴量値（時間平均）を保持
    shap_samples_list: List[np.ndarray] = []
    feature_values_list: List[np.ndarray] = []

    for fold_idx, test_sid in enumerate(SUBJECT_IDS):
        print(f"\n[INFO][SHAP] ===== LOSO fold {fold_idx+1}/{len(SUBJECT_IDS)}: Test {test_sid} =====")

        # 学習データ（LOSO）
        train_X_list = []
        train_y_list = []
        for sid in SUBJECT_IDS:
            if sid == test_sid:
                continue
            train_X_list.append(X_by_sid[sid])
            train_y_list.append(y_by_sid[sid])

        train_X = np.concatenate(train_X_list, axis=0)  # (N_train, SEQ_LEN, N_FEATURES)
        train_y = np.concatenate(train_y_list, axis=0)  # (N_train,)

        print(
            f"[INFO][SHAP] Fold data sizes: "
            f"Train N_seq={len(train_y)}, Test N_seq={len(y_by_sid[test_sid])}"
        )

        # --- モデル学習 ---
        model = train_one_fold_for_shap(train_X, train_y, device=device)
        model.eval()

        # --- SHAP用の背景データ（background）をサンプリング ---
        n_train = train_X.shape[0]
        if n_train > N_BACKGROUND_MAX:
            idx_bg = np.random.choice(n_train, size=N_BACKGROUND_MAX, replace=False)
            bg_X = train_X[idx_bg]
        else:
            bg_X = train_X

        print(f"[INFO][SHAP] Using {len(bg_X)} samples as background")
        background = torch.from_numpy(bg_X).float().to(device)

        # --- SHAPを計算する対象サンプル（訓練シーケンスのサブセット） ---
        if n_train > N_SHAP_EVAL_MAX:
            idx_eval = np.random.choice(n_train, size=N_SHAP_EVAL_MAX, replace=False)
            X_eval = train_X[idx_eval]
        else:
            X_eval = train_X

        print(f"[INFO][SHAP] Computing SHAP on {len(X_eval)} training sequences")
        X_eval_tensor = torch.from_numpy(X_eval).float().to(device)

        # --- DeepExplainer を構築 ---
        wrapped_model = WrappedLSTMForSHAP(model).to(device)
        explainer = shap.DeepExplainer(wrapped_model, background)

        # shap_values: (N_eval, SEQ_LEN, N_FEATURES, [output_dim]) か，
        # それを要素に持つリスト
        # ★ additivity チェックをオフ（RNN系でよく落ちるので）
        shap_values = explainer.shap_values(X_eval_tensor, check_additivity=False)

        # 戻り値の型に応じて整形
        if isinstance(shap_values, list):
            sv = shap_values[0]
        else:
            sv = shap_values

        if isinstance(sv, torch.Tensor):
            sv = sv.detach().cpu().numpy()
        else:
            sv = np.array(sv)

        # 出力次元が最後に1つだけ付いている場合は squeeze
        # 例: (N_eval, SEQ_LEN, N_FEATURES, 1) → (N_eval, SEQ_LEN, N_FEATURES)
        if sv.ndim == 4 and sv.shape[-1] == 1:
            sv = sv[..., 0]

        # 形チェック
        if sv.ndim != 3 or sv.shape[1] != SEQ_LEN or sv.shape[2] != N_FEATURES:
            raise RuntimeError(
                f"[ERROR][SHAP] Unexpected SHAP shape: {sv.shape}, "
                f"expected (N_eval, {SEQ_LEN}, {N_FEATURES})"
            )

        # --- beeswarm 用：時間方向で平均して 2次元にする ---
        # sv_mean_t: (N_eval, N_FEATURES) … 30秒履歴の平均寄与
        sv_mean_t = sv.mean(axis=1)

        # 特徴量値も時間平均をとる（色付け用）
        X_eval_np = X_eval_tensor.detach().cpu().numpy()  # (N_eval, SEQ_LEN, N_FEATURES)
        X_mean_t = X_eval_np.mean(axis=1)                 # (N_eval, N_FEATURES)

        shap_samples_list.append(sv_mean_t)
        feature_values_list.append(X_mean_t)

        # --- fold内の特徴重要度：mean(|SHAP|) over (samples, time) ---
        abs_sv = np.abs(sv)
        imp_fold = abs_sv.mean(axis=(0, 1))  # (N_FEATURES,)
        fold_importances.append(imp_fold)
        fold_meta.append({"fold_idx": fold_idx, "test_subject": test_sid})

        # foldごとのトップ特徴をざっくり表示
        order = np.argsort(-imp_fold)  # 降順
        top_k = min(5, N_FEATURES)
        print("[INFO][SHAP] Top features in this fold:")
        for i in range(top_k):
            j = order[i]
            print(f"  {i+1}. {FEATURE_COLS[j]} : mean|SHAP| = {imp_fold[j]:.4e}")

    # --- fold間で平均して最終重要度を算出 ---
    imp_mat = np.stack(fold_importances, axis=0)  # (n_folds, N_FEATURES)
    mean_imp = imp_mat.mean(axis=0)               # (N_FEATURES,)
    std_imp = imp_mat.std(axis=0)                 # (N_FEATURES,)

    order_global = np.argsort(-mean_imp)          # 降順

    # ランキング表を作成
    rows = []
    for rank, idx in enumerate(order_global, start=1):
        rows.append(
            {
                "rank": rank,
                "feature": FEATURE_COLS[idx],
                "mean_abs_shap": float(mean_imp[idx]),
                "std_abs_shap": float(std_imp[idx]),
            }
        )
    df_rank = pd.DataFrame(rows)

    # --- CSV保存（全体ランキング） ---
    rank_path = OUT_DIR / "Cell2_LSTM_SHAP_feature_importance.csv"
    df_rank.to_csv(rank_path, index=False)
    print(f"[INFO][SHAP] Saved SHAP feature ranking to: {rank_path}")

    # fold別の重要度行列も保存
    df_fold_imp = pd.DataFrame(
        imp_mat,
        columns=[f"SHAP_{name}" for name in FEATURE_COLS],
    )
    df_fold_imp.insert(0, "test_subject", [m["test_subject"] for m in fold_meta])
    df_fold_imp.insert(0, "fold_idx", [m["fold_idx"] for m in fold_meta])
    fold_imp_path = OUT_DIR / "Cell2_LSTM_SHAP_feature_importance_by_fold.csv"
    df_fold_imp.to_csv(fold_imp_path, index=False)
    print(f"[INFO][SHAP] Saved fold-wise SHAP importances to: {fold_imp_path}")

    # --- バー図で可視化（特徴重要度, mean|SHAP|） ---
    plt.figure(figsize=(8, 6))
    idxs = order_global  # 重要度降順
    y_pos = np.arange(len(idxs))

    plt.barh(y_pos, mean_imp[idxs])
    plt.yticks(y_pos, [FEATURE_COLS[i] for i in idxs], fontsize=20)
    plt.gca().invert_yaxis()

    plt.xlabel("Mean |SHAP| (over samples & time)", fontsize=24)
    plt.title("LSTM (30s history) SHAP feature importance", fontsize=30)
    plt.xticks(fontsize=20)

    plt.tight_layout()

    fig_path = OUT_DIR / "Cell2_LSTM_SHAP_feature_importance_bar.png"
    plt.savefig(fig_path, dpi=300)
    plt.close()
    print(f"[INFO][SHAP] Saved SHAP bar plot to: {fig_path}")

    # --- SHAP summary beeswarm 図（全fold統合） ---
    try:
        shap_all = np.concatenate(shap_samples_list, axis=0)   # (N_total, N_FEATURES)
        X_all_plot = np.concatenate(feature_values_list, axis=0)
        X_all_df = pd.DataFrame(X_all_plot, columns=FEATURE_COLS)

        shap.summary_plot(
            shap_all,
            X_all_df,
            feature_names=FEATURE_COLS,
            show=False,
            max_display=len(FEATURE_COLS),
        )

        fig = plt.gcf()
        ax = plt.gca()
        ax.set_xlabel("SHAP value (impact on model output)", fontsize=24)
        fig.suptitle("LSTM (30s history) SHAP summary (all folds)", fontsize=30, y=1.02)

        for lbl in ax.get_xticklabels():
            lbl.set_fontsize(20)
        for lbl in ax.get_yticklabels():
            lbl.set_fontsize(20)

        fig.tight_layout()

        beeswarm_path = OUT_DIR / "Cell2_LSTM_SHAP_summary_beeswarm.png"
        fig.savefig(beeswarm_path, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print(f"[INFO][SHAP] Saved SHAP summary beeswarm plot to: {beeswarm_path}")
    except Exception as e:
        print(f"[WARN][SHAP] Failed to create SHAP summary beeswarm plot: {e}")


if __name__ == "__main__":
    main()
